In [1]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter

In [2]:
path = "./Sales_Data"
files = [file for file in os.listdir(path) if not file.startswith('.')] # Ignore hidden files

all_months_data = pd.DataFrame()

for file in files:
    current_data = pd.read_csv(path+"/"+file)
    all_months_data = pd.concat([all_months_data, current_data])
    
all_months_data.to_csv("all_data_copy.csv", index=False)

In [3]:
all_data = pd.read_csv("all_data_copy.csv")
all_data

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186846,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186847,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [4]:
# Find NAN
nan_df = all_data[all_data.isna().any(axis=1)]
display(nan_df.head())

all_data = all_data.dropna(how='all')
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN,NaN,NaN
1433,NaN,NaN,NaN,NaN,NaN,NaN
1553,NaN,NaN,NaN,NaN,NaN,NaN


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"


In [5]:
all_data = all_data[all_data['Order Date'].str[0:2]!='Or']

In [6]:
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])

In [7]:
all_data['Month'] = all_data['Order Date'].str[0:2]
all_data['Month'] = all_data['Month'].astype('int32')
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4


In [8]:
def get_city(address):
    return address.split(",")[1].strip(" ")

def get_state(address):
    return address.split(",")[2].split(" ")[1]

all_data['City'] = all_data['Purchase Address'].apply(lambda x: f"{get_city(x)}  ({get_state(x)})")
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,City
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4,Dallas (TX)
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4,Boston (MA)
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA)


In [9]:
all_data.to_excel("output.xlsx", index = False)

In [10]:
df_ordered = all_data.groupby(['Order ID']).sum()
df_ordered

,Quantity Ordered,Price Each,Month
Order ID,,,
141234,1,700.00,1
141235,1,14.95,1
141236,2,11.99,1
141237,1,149.99,1
141238,1,11.99,1
...,...,...,...
319666,1,14.95,12
319667,2,3.84,12
319668,1,400.00,12


In [11]:
df_ordered.describe()

,Quantity Ordered,Price Each,Month
count,178437.000000,178437.000000,178437.000000
mean,1.171724,192.163793,7.356361
std,0.501673,341.032559,4.005809
min,1.000000,2.990000,1.000000
25%,1.000000,11.950000,4.000000
50%,1.000000,14.950000,7.000000
75%,1.000000,150.000000,10.000000
max,9.000000,3400.000000,48.000000


In [12]:
# https://stackoverflow.com/questions/43348194/pandas-select-rows-if-id-appear-several-time
df = all_data[all_data['Order ID'].duplicated(keep=False)]

# Referenced: https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
df2 = df[['Order ID', 'Grouped']].drop_duplicates()
df2

C:\Users\Ozan\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Order ID,Grouped
3,176560,"Google Phone,Wired Headphones"
18,176574,"Google Phone,USB-C Charging Cable"
30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586,"AAA Batteries (4-pack),Google Phone"
119,176672,"Lightning Charging Cable,USB-C Charging Cable"
...,...,...
186781,259296,"Apple Airpods Headphones,Apple Airpods Headphones"
186783,259297,"iPhone,Lightning Charging Cable,Lightning Char..."
186791,259303,"34in Ultrawide Monitor,AA Batteries (4-pack)"
186803,259314,"Wired Headphones,AAA Batteries (4-pack)"


In [14]:
count = Counter()

product1 = []
product2 = []
product3 = []
product4 = []

for row in df2['Grouped']:
    row_list = row.split(',')
    product1.append(row_list[0])
    product2.append(row_list[1])
    if len(row_list) > 2:
        product3.append(row_list[2])
    else:
        product3.append(np.nan)
    count.update(Counter(combinations(row_list, 2)))

In [15]:
df2 = df2.reset_index()
df2

,index,Order ID,Grouped
0,3,176560,"Google Phone,Wired Headphones"
1,18,176574,"Google Phone,USB-C Charging Cable"
2,30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
3,32,176586,"AAA Batteries (4-pack),Google Phone"
4,119,176672,"Lightning Charging Cable,USB-C Charging Cable"
...,...,...,...
7131,186781,259296,"Apple Airpods Headphones,Apple Airpods Headphones"
7132,186783,259297,"iPhone,Lightning Charging Cable,Lightning Char..."
7133,186791,259303,"34in Ultrawide Monitor,AA Batteries (4-pack)"
7134,186803,259314,"Wired Headphones,AAA Batteries (4-pack)"


In [17]:
product1 = np.array(product1)
product1 = pd.Series(product1, name = "Product 1")
df2 = df2.join(product1)
product2 = np.array(product2)
product2 = pd.Series(product2, name = "Product 2")
df2 = df2.join(product2)
product3 = np.array(product3)
product3 = pd.Series(product3, name = "Product 3")
df2 = df2.join(product3)
df2

,index,Order ID,Grouped,Product 1,Product 2,Product 3
0,3,176560,"Google Phone,Wired Headphones",Google Phone,Wired Headphones,nan
1,18,176574,"Google Phone,USB-C Charging Cable",Google Phone,USB-C Charging Cable,nan
2,30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea...",Bose SoundSport Headphones,Bose SoundSport Headphones,nan
3,32,176586,"AAA Batteries (4-pack),Google Phone",AAA Batteries (4-pack),Google Phone,nan
4,119,176672,"Lightning Charging Cable,USB-C Charging Cable",Lightning Charging Cable,USB-C Charging Cable,nan
...,...,...,...,...,...,...
7131,186781,259296,"Apple Airpods Headphones,Apple Airpods Headphones",Apple Airpods Headphones,Apple Airpods Headphones,nan
7132,186783,259297,"iPhone,Lightning Charging Cable,Lightning Char...",iPhone,Lightning Charging Cable,Lightning Charging Cable
7133,186791,259303,"34in Ultrawide Monitor,AA Batteries (4-pack)",34in Ultrawide Monitor,AA Batteries (4-pack),nan
7134,186803,259314,"Wired Headphones,AAA Batteries (4-pack)",Wired Headphones,AAA Batteries (4-pack),nan


In [18]:
df2_test = df2
df2_test = df2_test.drop(columns = ["Grouped", "index"])

In [19]:
df2_test
df2_test.to_csv("df_classification_copy.csv", index=False)

In [21]:
df_class = pd.read_csv("df_classification_copy.csv")
df_class

,Order ID,Product 1,Product 2
0,176560,Google Phone,Wired Headphones
1,176574,Google Phone,USB-C Charging Cable
2,176585,Bose SoundSport Headphones,Bose SoundSport Headphones
3,176586,AAA Batteries (4-pack),Google Phone
4,176672,Lightning Charging Cable,USB-C Charging Cable
...,...,...,...
7131,259296,Apple Airpods Headphones,Apple Airpods Headphones
7132,259297,iPhone,Lightning Charging Cable
7133,259303,34in Ultrawide Monitor,AA Batteries (4-pack)
7134,259314,Wired Headphones,AAA Batteries (4-pack)
